In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(
    temperature=0.1,
)

prompt = PromptTemplate.from_template("Who is the weather in {city}")
chain = prompt | llm
chain.invoke({"city": "rome"})

위와 같이 실시간 정보(특정 지역의 날씨)는 받아올 수 없기 때문에, 함수에 해당 정보를 넘겨줘야 함

In [4]:
def get_weather(lon, lat):
    print("call an api...")
    
function = {
    "name": "get_weather",
    "description": "function that takes longitude and latitude to find the weather of a place",
    "parameters": {
        "type": "object",
        "properties": {
                "lon": {
                    "type": "string",
                    "description": "The longitude coordinate"
                },
                "lat": {
                    "type": "string",
                    "description": "The latitude coordinate"
                },
        }
    },
    "required": ["lon", "lat"]
}

In [11]:
llm = ChatOpenAI(
    temperature=0.1,
).bind(function_call="auto", functions=[function])

prompt = PromptTemplate.from_template("Who is the weather in {city}")
chain = prompt | llm
response = chain.invoke({"city": "rome"})
response = response.additional_kwargs["function_call"]["arguments"]
response

'{\n  "lon": "12.4964",\n  "lat": "41.9028"\n}'

In [10]:
import json

r = json.loads(response)
get_weather(r["lon"], r["lat"])

call an api...


In [12]:
function = {
    "name": "create_quiz",
    "description": "function that takes a list of questions and answers and returns a quiz",
    "parameters": {
        "type": "object",
        "properties": {
            "questions": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "question": {
                            "type": "string",
                        },
                        "answers": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "answer": {
                                        "type": "string",
                                    },
                                    "correct": {
                                        "type": "boolean",
                                    },
                                },
                                "required": ["answer", "correct"],
                            },
                        },
                    },
                    "required": ["question", "answers"],
                },
            }
        },
        "required": ["questions"],
    },
}

In [13]:
llm = ChatOpenAI(
    temperature=0.1,
).bind(
    function_call={
        "name": "create_quiz",
    },
    functions=[
        function,
    ],
)

In [14]:
prompt = PromptTemplate.from_template("Make a quiz about {city}")
chain = prompt | llm
response = chain.invoke({"city": "rome"})
response = response.additional_kwargs["function_call"]["arguments"]

response

'{\n  "questions": [\n    {\n      "question": "What year was Rome founded?",\n      "answers": [\n        {\n          "answer": "753 BC",\n          "correct": true\n        },\n        {\n          "answer": "476 AD",\n          "correct": false\n        },\n        {\n          "answer": "1492 AD",\n          "correct": false\n        },\n        {\n          "answer": "1066 AD",\n          "correct": false\n        }\n      ]\n    },\n    {\n      "question": "Who was the first emperor of Rome?",\n      "answers": [\n        {\n          "answer": "Julius Caesar",\n          "correct": false\n        },\n        {\n          "answer": "Augustus",\n          "correct": true\n        },\n        {\n          "answer": "Nero",\n          "correct": false\n        },\n        {\n          "answer": "Constantine",\n          "correct": false\n        }\n      ]\n    },\n    {\n      "question": "What was the official language of the Roman Empire?",\n      "answers": [\n        {\n     

In [15]:
json.loads(response)

{'questions': [{'question': 'What year was Rome founded?',
   'answers': [{'answer': '753 BC', 'correct': True},
    {'answer': '476 AD', 'correct': False},
    {'answer': '1492 AD', 'correct': False},
    {'answer': '1066 AD', 'correct': False}]},
  {'question': 'Who was the first emperor of Rome?',
   'answers': [{'answer': 'Julius Caesar', 'correct': False},
    {'answer': 'Augustus', 'correct': True},
    {'answer': 'Nero', 'correct': False},
    {'answer': 'Constantine', 'correct': False}]},
  {'question': 'What was the official language of the Roman Empire?',
   'answers': [{'answer': 'Latin', 'correct': True},
    {'answer': 'Greek', 'correct': False},
    {'answer': 'Italian', 'correct': False},
    {'answer': 'Spanish', 'correct': False}]},
  {'question': 'Who was the legendary founder of Rome?',
   'answers': [{'answer': 'Romulus', 'correct': True},
    {'answer': 'Remus', 'correct': False},
    {'answer': 'Caesar', 'correct': False},
    {'answer': 'Hannibal', 'correct': Fal